In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
file_list = drive.ListFile({'q': "'13uceITtEknqn4WiEC9fvpttTd9mn8mSe' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

In [0]:
train_downloaded = drive.CreateFile({'id': '106i86-VEOKrKWdT6xRHmX2tVfIe413rq'})
train_downloaded.GetContentFile('train.csv')
test_downloaded = drive.CreateFile({'id': '1prpU3Z6pHHRpB5T4XvYBPkGzcz3FMU8a'})
test_downloaded.GetContentFile('test.csv')
sample_downloaded = drive.CreateFile({'id': '1NnSvlLJTxQJnGGKHgbT8DYN7jWqQaaQl'})
sample_downloaded.GetContentFile('sample.csv')

ApiRequestError: ignored

In [0]:
import warnings; warnings.simplefilter('ignore')
import pickle
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
import numpy as np 
import pandas as pd 
import time
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import pandas as pd 
import re
import numpy as np
#from gensim.models import Doc2Vec, doc2vec
from nltk.corpus import stopwords
#from textblob import TextBlob, Word
import nltk
import pandas as pd
import tensorflow as tf
from keras import backend
backend.tensorflow_backend._get_available_gpus()
import nltk
nltk.download('wordnet')
nltk.download('stopwords')


Using TensorFlow backend.


[nltk_data] Downloading package wordnet to /content/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /content/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
tick = time.time()
df = pd.read_csv("train.csv", dtype={"label": object, "text": object})
y = df.label 
text = df.text

print(df)
#Convert all text to lower case
text = text.apply(lambda x: " ".join(x.lower() for x in str(x).split()))

#Remove all punctuation
for i in range(0, len(text)):
    text[i] = re.sub(r'[^\w\s]', "", text[i])
    
#for index,row in df.iterrows():
#    if df.iloc[index]['text'] == 'My husband and I had not purchased a home before and we definitely needed some hand holding. They were patient and professional. We got our dream home and the entire experience was awesome! Thank you so much ladies for a job well done!':
#        df.drop(index, inplace=True)

#Remove stop words (removes important words! needs modification)
stop = stopwords.words("english")
stop = stop[:143]
stop.remove("not")
stop.remove("against")
stop.remove("no")
#stop.append("My husband and I had not purchased a home before and we definitely needed some hand holding. They were patient and professional. We got our dream home and the entire experience was awesome! Thank you so much ladies for a job well done! ")
text = text.apply(lambda x: " ".join(x for x in str(x).split() if x not in stop))

#Lemmatize (not working)
#text = text.apply(lambda x: " ".join(Word(x).lemmatize() for x in x.split()))

#Stemming

#text.drop("My husband and I had not purchased a home before and we definitely needed some hand holding. They were patient and professional. We got our dream home and the entire experience was awesome! Thank you so much ladies for a job well done!")
#seperating words into lists
for i, line in enumerate(text):
    text[i] = line.split()
#print(text)
#df.text = text
#text = list(text)
print(time.time() - tick)

FileNotFoundError: ignored

In [0]:
X = text

Y = df['label']
X

0         [great, mobile, app, nice, reward, program, ma...
1         [really, fast, polite, definitely, recommend, ...
2         [place, always, amazing, friendly, staff, grea...
3         [wine, 101, class, friday, night, cool, spot, ...
4         [rounding, think, place, may, potential, coupl...
5         [romantic, tryst, planet, hollywood, poker, ro...
6         [got, color, haircut, finally, months, looking...
7         [never, go, place, nearly, empty, 430, pm, sat...
8         [ive, come, 2, years, nps, bought, staff, alwa...
9         [07082014, war, ich, mit, 5, kollegen, mittags...
10        [5, stars, takeout, not, recommended, dinein, ...
11        [nachdem, wir, jetzt, das, dritte, mal, vor, d...
12        [hands, best, polish, boy, sandwich, cleveland...
13        [oh, fuzzys, begin, familiar, toby, keith, son...
14        [ive, tried, many, cupcakes, magnolia, prarie,...
15        [put, irish, pub, walmart, youd, get, dont, ge...
16        [amazing, pho, almost, drinkin

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [0]:
max_features = 20000
#tokenizer = Tokenizer(num_words=max_features)
tokenizer = Tokenizer(num_words=max_features, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True,split=' ')
tokenizer.fit_on_texts(list(X_train))
list_tokenized_train = tokenizer.texts_to_sequences(X_train)
list_tokenized_test = tokenizer.texts_to_sequences(X_test)


In [0]:
X = tokenizer.texts_to_sequences(text)
X = pad_sequences(X)


Y = pd.get_dummies(y.values)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)
print(y.values)
print(Y_train.values)


(320000, 571) (320000, 3)
(80001, 571) (80001, 3)
['1' '2' '2' ... '2' '2' '0']
[[0 0 1]
 [1 0 0]
 [0 0 1]
 ...
 [0 0 1]
 [0 0 1]
 [0 0 1]]


In [0]:
maxlen = 100
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [0]:
Y_train, Y_test = train_test_split(Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)
print(y.values)

(320000, 571) (320000, 3)
(80001, 571) (80001, 3)
['1' '2' '2' ... '2' '2' '0']


In [0]:


inp = Input(shape=(maxlen, ))
embed_size = 128
x = Embedding(max_features, embed_size)(inp)
x = LSTM(200, return_sequences=True,name='lstm_layer')(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.1)(x)
x = Dense(120, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(60, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(4, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 100, 128)          2560000   
_________________________________________________________________
lstm_layer (LSTM)            (None, 100, 200)          263200    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 200)               0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 120)               24120     
_________________________________________________________________
dropout_12 (Dropout)         (None, 120)               0         
__________

In [0]:
print("start fitting...")
model.fit(X_t, y_train , epochs=3, batch_size=32, validation_split=0.1)

start fitting...


ValueError: ignored

In [0]:
scores = model.evaluate(X_te, y_test)
print("\n%s: %.4f%%" % (model.metrics_names[1], scores[1]*100))
y_pred = model.predict(X_te, batch_size=1024)
y_classes = y_pred.argmax(axis=-1)

80001/80001 [==============================] - 146s 2ms/step

acc: 90.4376%


In [0]:
z=np.array(y_test)
d=y_test.astype(int)
d
z=np.array(d)
z[0]
Y_TEST=z

In [0]:

accScore = metrics.accuracy_score(Y_TEST,y_classes)

lbl = [0,1,2]
precision = metrics.precision_score(Y_TEST,y_classes,average=None,labels=lbl)
recall = metrics.recall_score(Y_TEST,y_classes,average=None,labels=lbl)
f1Score = metrics.f1_score(Y_TEST,y_classes,average=None,labels=lbl)

print("\nOverall Acurracy: ",accScore,"\n")

for i in range(len(lbl)):
    print("Precision of %s class: %f" %(lbl[i],precision[i]))
    print("Recall of %s class: %f" %(lbl[i],recall[i]))
    print("F1-Score of %s class: %f" %(lbl[i],f1Score[i]),"\n")



Overall Acurracy:  0.9043761952975587 

Precision of 0 class: 0.910162
Recall of 0 class: 0.876813
F1-Score of 0 class: 0.893176 

Precision of 1 class: 0.780361
Recall of 1 class: 0.712560
F1-Score of 1 class: 0.744921 

Precision of 2 class: 0.932406
Recall of 2 class: 0.965425
F1-Score of 2 class: 0.948628 



In [0]:
import utils

ModuleNotFoundError: ignored

In [0]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding, Flatten
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
kernel_size=3
embedding_matrix = np.random.randn(20000, 128) * 0.01

In [0]:
inp = Input(shape=(100, ))
embed_size = 128
x = Embedding(20000,128,weights=[embedding_matrix])(inp)#input_length=maxlen)(inp)
x=(Dropout(0.4))(x)
x=(Conv1D(600, kernel_size, padding='valid', activation='relu', strides=1))(x)
x=(Conv1D(300, kernel_size, padding='valid', activation='relu', strides=1))(x)
x=(Conv1D(150, kernel_size, padding='valid', activation='relu', strides=1))(x)
x=(Conv1D(75, kernel_size, padding='valid', activation='relu', strides=1))(x)
x=(Flatten())(x)
x=(Dense(600))(x)
x=(Dropout(0.5))(x)
x=(Activation('relu'))(x)
x=(Dense(1))(x)
x=(Activation('softmax'))(x)
model=Model(inputs=inp, outputs=x)

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 100, 128)          2560000   
_________________________________________________________________
dropout_9 (Dropout)          (None, 100, 128)          0         
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 98, 600)           231000    
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 96, 300)           540300    
_________________________________________________________________
conv1d_19 (Conv1D)           (None, 94, 150)           135150    
_________________________________________________________________
conv1d_20 (Conv1D)           (None, 92, 75)            33825     
__________